# 2장 고급 스크레이핑
2-1 로그인이 필요한 사이트에서 다운받기

#### login-getmileage

In [2]:
# 로그인을 위한 모듈 추출하기
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# 아이디와 비밀번호 지정하기 -- (*1)
USER = "dkssud8150"
PASS = "dkssud881!"

# 세션 시작하기 -- (*2)
session = requests.session()
# rsquests.session() = session을 생성

# 로그인하기 -- (*3)
login_info = { # parameter 형태 => 자료형 이름 = {"param1":"value1","param2":"value2",...}
    "m_id": USER, # 아이디 지정, html에서 'name:m_id'로 된 것을 불러옴 = post형식
    "m_passwd": PASS # 비밀번호 지정
}
# parameter로 직접 넣어서 보내기 => url="http://www.naver.com?a=b&bb=123", res=requests.get(url)
# dictionary 사용 => paramDict = {"a":"b","bb":123} , url="http://www.naver.com", res=requests.get(url, params=paramDict)

url_login = "https://www.hanbit.co.kr/member/login_proc.php"
res = session.post(url_login, data=login_info) # POST방식으로 된 데이터 요청, post로 넣을때는 파라미터를 넣지 않아서 dictionary 사용해야 함
# data를 딕셔너리 구조로 유지하면서 문자열로 바꿔서 전달해야 한다. 이때 사용하는 것이 json 모듈 => 
# json.dumps() {s를 빼먹으면 오류남} = json 포멧 데이터로 바꿔서 파이썬에서 계속 작업하기 위함.
res.raise_for_status() # 오류가 발생하면 예외가 발생합니다.
# res.request = 내가 보낸 request 객체에 접근 가능, res.status_code = 응답 코드, 
# res.raise_for_status = 200 ok 코드가 아닌 경우 에러 발동, res.json = json response일 경우 딕셔너리 타입으로 바로 변환

# 마이페이지에 접근하기 -- (*4)
url_mypage = "https://www.hanbit.co.kr/myhanbit/myhanbit.html" # 마이페이지 주소
res = session.get(url_mypage)
res.raise_for_status()

# 마일리지와 이코인 가져오기 -- (*5)
soup = BeautifulSoup(res.text, "html.parser")
mileage = soup.select_one(".mileage_section1 span").get_text() 
# get_text = 현재 태그를 포함하여 모든 하위 태그를 제거하고 유니코드 텍스트만 들어있는 문자열을 반환
ecoin = soup.select_one(".mileage_section2 span").get_text()
print("마일리지: " + mileage)
print("이코인: " + ecoin)

마일리지: 2,000
이코인: 0


#### requests-test

In [3]:
# 데이터 가져오기
import requests
r = requests.get("http://api.aoikujira.com/time/get.php")

# 텍스트 형식으로 데이터 추출하기
text = r.text
print(text) # 일반 텍스트

# 바이너리 형식으로 데이터 추출하기
bin = r.content
print(bin) # b''형태 => b''는 파이썬에서 바이너리라는 것을 나타냄

2020/08/24 04:39:30
b'2020/08/24 04:39:30'


#### request-png

In [4]:
# 바이너리를 활용해서 바이너리 데이터인 이미지를 받아 저장하는 예제
# 이미지 데이터 추출하기
import requests
r = requests.get("http://wikibook.co.kr/wikibook.png")

# 바이너리 형식으로 데이터 저장하기
with open("test.png", "wb") as f: # 쓰기 형태의 바이너리 파일로 연다.
    f.write(r.content) # content = 바이트로 리턴함
    
print("saved")

saved


# 2장 고급 스크레이핑
2-2 웹 브라우저를 이용한 스크레이핑

# 2장 고급 스크레이핑
2-3 웹 API로 데이터 추출하기

#### api-weather

In [20]:
import requests
import json

# API 키를 지정합니다. 자신의 키로 변경해서 사용해주세요 -- (*1)
apikey = "bd3b4fc8e2e1a5186b7049905ced6afb"

# 날씨를 확인할 도시 지정하기 -- (*2)
cities = ["Seoul,KR", "Tokyo,JP", "New York,US"] # [도시이름, 국가이름]

# API 지정 -- (*3)
api = "http://api.openweathermap.org/data/2.5/weather?q={city}&APPID={key}" 
# api의 기본적인 형태를 지정
# 매개변수 q에 도시이름, appid에 api키를 지정
# 파이썬에서는 "xxx{name}xxx"와 같은 형식으로 문자열을 만들고 format() 메서드로 {name} 부분을 다른 문자열로 대체할 수 있습니다.

# 켈빈 온도를 섭씨 온도로 변환하는 함수 -- (*4)
k2c = lambda k: k - 273.15

# 각 도시의 정보 추출하기 --(*5)
for name in cities: 
    # API의 URL 구성하기 -- (*6)
    url = api.format(city=name, key=apikey)
    # API에 요청을 보내 데이터 추출하기
    r = requests.get(url)
    
    # 결과를 JSON 형식을 파이썬 데이터로 변환하기 -- (*7)
    data = json.loads(r.text)
    
    # 결과 출력하기 -- (*8)
    print("+ 도시 =", data["name"])
    print("| 날씨 =", data["weather"][0]["description"])
    print("| 최저 기온 =", k2c(data["main"]["temp_min"]),"ºc")
    print("| 최고 기온 =", k2c(data["main"]["temp_max"]),"ºc")
    print("| 습도 =", data["main"]["humidity"])
    print("| 기압 =", data["main"]["pressure"])
    print("| 풍향 =", data["wind"]["deg"])
    print("| 풍속 =", data["wind"]["speed"])
    print("")

+ 도시 = Seoul
| 날씨 = overcast clouds
| 최저 기온 = 23.0 ºc
| 최고 기온 = 25.0 ºc
| 습도 = 83
| 기압 = 1006
| 풍향 = 86
| 풍속 = 0.69

+ 도시 = Tokyo
| 날씨 = shower rain
| 최저 기온 = 25.0 ºc
| 최고 기온 = 26.110000000000014 ºc
| 습도 = 94
| 기압 = 1007
| 풍향 = 40
| 풍속 = 3.6

+ 도시 = New York
| 날씨 = few clouds
| 최저 기온 = 28.890000000000043 ºc
| 최고 기온 = 31.110000000000014 ºc
| 습도 = 62
| 기압 = 1017
| 풍향 = 170
| 풍속 = 4.1

